In [1]:
from torch.utils.cpp_extension import load

trisum = load(name='trisum_cuda', sources=['trisum_cuda.cpp', 'trisum_cuda_kernel.cu'])


In [2]:
import torch

In [3]:
dir(trisum)

['__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'backward',
 'forward']

In [4]:
from trisumt import trisum

In [5]:
sz = 16
idxs = torch.arange(sz).repeat(3).reshape(1,3*sz).to(torch.int32)
int(idxs[0,1])

1

In [6]:
from torch import nn

bs = 16*256
sz = 256
device = torch.device("cuda:1")
rnn = trisum(sz).to(device)
X =  torch.arange(bs*sz).reshape(bs,sz).to(torch.float32).to(device)
idxs = torch.arange(sz).repeat(3*sz).reshape(1,3*sz**2).to(torch.int32).to(device)
idxs = torch.cat([idxs,torch.flip(idxs,[1])],dim = 0)



In [7]:
#print(idxs.shape)
#print(idxs[0,0:5],idxs[1,0:5])
for _ in range(100):
    output = rnn(X,idxs,idxs)
print(output[0])

tensor([[ -55.1102,    1.3186, -289.5669,  ...,  -91.2199,  112.6162,
          -24.7189],
        [ -55.1102,    1.3186, -289.5669,  ...,  -91.2199,  112.6162,
          -24.7189],
        [ -55.1102,    1.3186, -289.5669,  ...,  -91.2199,  112.6162,
          -24.7189],
        ...,
        [ -55.1102,    1.3186, -289.5669,  ...,  -91.2199,  112.6162,
          -24.7189],
        [ -55.1102,    1.3186, -289.5669,  ...,  -91.2199,  112.6162,
          -24.7189],
        [ -55.1102,    1.3186, -289.5669,  ...,  -91.2199,  112.6162,
          -24.7189]], device='cuda:1')


In [8]:
idxs2 = idxs.clone().to(torch.long)
for _ in range(100):
    y = X[:,idxs2[0]] - X[:,idxs2[1]]
    #indexing to maintain Rotation and Translation invariance, Break Mirror invariance
    weights = rnn.weights.data
    y = (y*weights.view(1,3*sz**2)).reshape((bs, sz, sz,3)).sum(-1).sum(-1)
print(y)

tensor([[ -55.1102,    1.3187, -289.5668,  ...,  -91.2201,  112.6162,
          -24.7189],
        [ -55.1102,    1.3187, -289.5668,  ...,  -91.2201,  112.6162,
          -24.7189],
        [ -55.1102,    1.3187, -289.5668,  ...,  -91.2201,  112.6162,
          -24.7189],
        ...,
        [ -55.1102,    1.3187, -289.5668,  ...,  -91.2201,  112.6162,
          -24.7189],
        [ -55.1102,    1.3187, -289.5668,  ...,  -91.2201,  112.6162,
          -24.7189],
        [ -55.1102,    1.3187, -289.5668,  ...,  -91.2201,  112.6162,
          -24.7189]], device='cuda:1')


In [9]:
rnn.weights.data

tensor([ 0.0478,  0.0519, -0.0146,  ...,  0.0031,  0.0192, -0.0023],
       device='cuda:1')

In [10]:
#Custom backward pass check
import math

class TriangSum(nn.Module):
    def __init__(self,  state_size):
        super(TriangSum, self).__init__()

        self.state_size = state_size
        self.weights = nn.Parameter(torch.Tensor(3*state_size**2))

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.state_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, +stdv)

    def forward(self, x, idxs):
        y = x[:,idxs[0]] - x[:,idxs[1]]
        weights = rnn.weights.data
        y = (y*self.weights.view(1,3*sz**2)).reshape((bs, sz, sz,3)).sum(-1).sum(-1)
        
        return y



In [11]:

bs = 16*256
sz = 256
device = torch.device("cuda:1")
rnn = trisum(sz).to(device)
X =  torch.tensor(torch.arange(bs*sz).reshape(bs,sz).to(torch.float32).to(device), requires_grad=True)
idxs = torch.arange(sz).repeat(3*sz).reshape(1,3*sz**2).to(torch.int32).to(device)
idxs = torch.cat([idxs,torch.flip(idxs,[1])],dim = 0)

/tmp/ipykernel_47916/3546125513.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X =  torch.tensor(torch.arange(bs*sz).reshape(bs,sz).to(torch.float32).to(device), requires_grad=True)


In [12]:
TriaSum = TriangSum(sz).to(device)
idxs2 = idxs.clone().to(torch.long)
Q = TriaSum(X, idxs2)
external_grad = torch.ones(Q.shape).to(device)
#external_grad = torch.rand(Q.shape).to(device)
Q.backward(gradient=external_grad)
print("X.grad", X.grad)
print("weights.grad", TriaSum.weights.grad)
print("X.grad shape", X.grad.shape)
print("weights.grad shape ", TriaSum.weights.grad.shape)

X.grad tensor([[ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        ...,
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409]],
       device='cuda:1')
weights.grad tensor([-1044480., -1036288., -1028096.,  ...,  1028096.,  1036288.,
         1044480.], device='cuda:1')
X.grad shape torch.Size([4096, 256])
weights.grad shape  torch.Size([196608])


In [13]:
idxs.shape

torch.Size([2, 196608])

In [14]:
def make_idxs_backward(idxs, num_patches):
    dim = 3*num_patches**2
    idxs_bw = torch.zeros((4,dim)).to(torch.long)
    ct = 0
    ct2 = 0
    for i in range(num_patches):
        for j in range(dim):
            if idxs[0][j] == i:
                c = j//(3*num_patches)
                idxs_bw[0][ct] = c
                idxs_bw[2][ct] = j 
                ct+=1
            if idxs[1][j] == i:
                c = j//(3*num_patches)
                idxs_bw[1][ct2] = c
                idxs_bw[3][ct2] = j 
                ct2+=1
    return torch.tensor(idxs_bw).to(torch.int32)


def make_idxs_backward2(idxs, num_patches):
    dim = 3*num_patches**2
    idxs_bw = torch.zeros((4,dim)).to(torch.long)
    ct = 0
    ct2 = 0
    for i in range(num_patches):
        idx = (idxs[0] == i).nonzero(as_tuple=True)[0].tolist()
        for j in idx:
            if idxs[0][j] == i:
                c = j//(3*num_patches)
                idxs_bw[0][ct] = c
                idxs_bw[2][ct] = j 
                ct+=1
        idx = (idxs[1] == i).nonzero(as_tuple=True)[0].tolist()
        for j in idx:
            if idxs[1][j] == i:
                c = j//(3*num_patches)
                idxs_bw[1][ct2] = c
                idxs_bw[3][ct2] = j 
                ct2+=1
    return torch.tensor(idxs_bw).to(torch.int32)
            
idxs_bw = make_idxs_backward2(idxs,sz).to(device)     
            
#idxs_bw = make_idxs_backward(idxs,sz).to(device)            
        

/tmp/ipykernel_47916/3070663716.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(idxs_bw).to(torch.int32)


In [15]:
idxs_bw.shape

torch.Size([4, 196608])

In [16]:
import trisum_cuda
variables = [X,idxs,idxs_bw,TriaSum.weights.data]
print(variables[1].shape,variables[-1].shape,)
grad_X, grad_weights = trisum_cuda.backward(external_grad.contiguous(), *variables)
print("grad_X", grad_X)
print("grad_weights", grad_weights)


torch.Size([2, 196608]) torch.Size([196608])
grad_X tensor([[ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        ...,
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409],
        [ 0.8409,  2.2706, -0.2784,  ...,  0.2784, -2.2706, -0.8409]],
       device='cuda:1')
grad_weights tensor([-1044480., -1036288., -1028096.,  ...,  1028096.,  1036288.,
         1044480.], device='cuda:1')


In [17]:
print("Comparison to Autograd")
print("X grad" ,torch.norm(grad_X-X.grad))
print("weights grad", torch.norm(grad_weights - TriaSum.weights.grad.data))

Comparison to Autograd
X grad tensor(0.0010, device='cuda:1')
weights grad tensor(0., device='cuda:1')


In [18]:
test = grad_weights - TriaSum.weights.grad.data
print(test)        

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:1')


In [ ]:
#Pre-defined gradcehk does ontt work well when indexing is used

In [ ]:
from torch.autograd import gradcheck
from trisumt import trisumFunction

In [ ]:
#X = torch.nn.Parameter(2.0*torch.ones( 3, 20)).double().to(device)
#W = torch.nn.Parameter(torch.rand(20)).double().to(device)
bs = 16
sz = 20
from torch.autograd import Variable
X = Variable(torch.rand( bs, sz).double().to(device),requires_grad=True)
W =  Variable(torch.rand(sz).double().to(device),requires_grad=True)
idxs = torch.arange(sz).repeat(3*sz).reshape(1,3*sz**2).to(torch.int32).to(device)
idxs =  Variable(torch.cat([idxs,torch.flip(idxs,[1])],dim = 0),requires_grad=False)
variables = [X, W, idxs]

gradcheck(trisumFunction.apply, variables) # eps=1e-6, atol=1e-4)


